Imports e Configuração

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

sns.set_theme(style="whitegrid")
%matplotlib inline

Carregar Dados Reais (do GitHub)

Preparar os Dados (Feature Selection)

Dividir em Treino e Teste

Treinar o Modelo (Linear Regression)

Avaliar e Visualizar